In [1]:
%cd ./bento

/home/jaemin/spmm_bento/bento


In [2]:
import io
from typing import List, Optional
import bentoml
import numpy as np
from bentoml.io import Text, NumpyNdarray, JSON
from pydantic import BaseModel

spmm_model = bentoml.models.get("spmm:latest").to_runner()
get_pv_and_mask = bentoml.models.get("get_pv_and_mask:latest").to_runner()

svc = bentoml.Service(
    name="spmm", runners=[spmm_model,get_pv_and_mask]
)


@svc.api(input=Text(), output=NumpyNdarray())
async def SMILES_to_PV(smiles: str) -> np.ndarray:
    pv_numpy = await spmm_model.async_run(mode='s2p', s2p_smiles=smiles)
    return pv_numpy

class PV2SmilesDto(BaseModel):
    property_names: List[str]
    value: List[float]
    num_samples: Optional[int] = 5

@svc.api(input=JSON(pydantic_model=PV2SmilesDto), output=Text())
async def PV_to_SMILES(dto:PV2SmilesDto) -> str:
    property_name = dto.property_names
    value = dto.value
    d = await get_pv_and_mask.async_run(property_name, value)
    pv = d["pv"]
    mask = d["mask"]
    smiles_list = await spmm_model.async_run(mode='p2s', p2s_prop=pv, p2s_mask=mask, stochastic=True, k=2, n_sample=dto.num_samples)
    return ', '.join(smiles_list)


/home/jaemin/.pyenv/versions/3.10.12/envs/bento-env-310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
for runner in svc.runners:
    runner.init_local()

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.
'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


In [4]:
# [INPUT] smiles: str
inp = {"smiles": 'c1ccccc1',}

result3 = await svc.apis["SMILES_to_PV"].func(**inp)
# [OUTPUT] result3: np.ndarray of shape(1, 53). The chemical property that each element correspond to is written in property_name.txt

SMILES-to-PV generation...


In [5]:
result3

array([[ 1.31968832e+00, -2.09236816e+02,  1.37784004e+00,
         4.00860596e+00,  1.30901432e+00,  1.62245846e+00,
         6.92060947e-01,  8.74613762e-01,  8.86291504e-01,
        -1.75620604e+00, -3.17974567e-01, -1.19450760e+01,
        -4.86771107e-01, -3.77045135e+01,  8.11459045e+01,
         7.19506085e-01,  1.02144337e+00,  1.21226382e+00,
         4.68280315e-02, -2.35983968e-01,  1.87203026e+01,
         7.79028015e+01,  2.58636379e+00,  1.00324154e-01,
        -9.78123188e-01,  1.20286713e+01,  5.55901814e+00,
         3.31680298e+00, -3.55727106e-01, -6.55701399e-01,
        -3.12857628e-02,  8.38295746e+00,  9.64353027e+01,
        -1.10602570e+00, -1.34027624e+00, -1.90883666e-01,
         4.92696762e-02,  4.21727896e-02, -5.47247171e-01,
         5.64566016e-01,  6.14910007e-01, -7.13792801e-01,
         2.00002313e-01, -7.48727798e-01,  6.36517815e-03,
        -1.27293491e+00, -1.21113956e-02,  1.45430267e-02,
         1.53303146e-02,  8.86715698e+00,  1.04305196e+0

In [6]:
p2i = dict()
with open('./property_name.txt', 'r') as f:
    lines = f.readlines()
for i, l in enumerate(lines):
    p2i[l.strip()] = i

# pv = calculate_property('COc1cccc(NC(=O)CN(C)C(=O)COC(=O)c2cc(c3cccs3)nc3ccccc23)c1').cpu().numpy()
# mask = np.zeros(53)

pv = np.zeros(53)
pv[p2i['ExactMolWt']] = 150.# ExactMolWt=property with the index 14
mask = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# [INPUT] pv: np.ndarray of shape (53,), mask: np.ndarray of shape (53,). The mask should only contain 0 and 1, where 1 means that property input will be ignored.
inp = {"property_names":['ExactMolWt'], "value":[150.] }
result1 = await svc.apis["PV_to_SMILES"].func(PV2SmilesDto(**inp))
# [OUTPUT] result1: str. This string contains 5 SMILES output, concatenated with comma.

PV-to-SMILES generation...


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00,  7.19it/s]


In [7]:
result1

'COC1=C(C)NC(C(C)C)=C1, N#CC=CC1CCCC1, CCC=CC1CCC(Cl)CC1, CNC(=S)NC1CCCC1, CC(C)CC1OC1CCCl'

: 